In [63]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools
import tensorflow as tf

from tensorflow.keras.preprocessing import image as kp_image
from tensorflow.keras import models 
from tensorflow.keras import losses
from tensorflow.keras import layers
from tensorflow.keras import backend as K



In [ ]:
content_path = '/content/WIN_20200714_10_41_07_Pro.jpg'
style_path = '/content/style.jpg'

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

def load_and_process_image(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img
def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [ ]:
vgg=keras.applications.vgg19.VGG19()

In [ ]:
import numpy as np
import random

In [ ]:
a=np.random.rand(5,5,3)

In [ ]:
a=tf.convert_to_tensor(a)

In [ ]:
a

In [ ]:
def fun(x):
  channels = int(x.shape[-1])
  print(channels)
  a = tf.reshape(x, [-1, channels])
  print(a.shape)
   



In [ ]:
fun(a)

In [ ]:
content_layers=['block5_conv2']
style_layers=[
              'block1_conv1',
              'block2_conv1',
              'block3_conv1',
              'block4_conv1',
              'block5_conv1'
]
num_content_layers=len(content_layers)
num_style_layers=len(style_layers)

In [ ]:
def get_model_():
  """ Creates our model with access to intermediate layers. 
  
  This function will load the VGG19 model and access the intermediate layers. 
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model. 
  
  Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
  """
  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  # Get output layers corresponding to style and content layers 
  style_outputs = [vgg.get_layer(name).output for name in style_layers]
  content_outputs = [vgg.get_layer(name).output for name in content_layers]
  model_outputs = style_outputs + content_outputs
  # Build model 
  return models.Model(vgg.input, model_outputs)

In [ ]:
def get_model():
  vgg=keras.applications.vgg19.VGG19(include_top=False,weights='imagenet')
  vgg.trainable=False
  style_outputs=[]
  content_outputs=[]
  for layers in style_layers:
    style_outputs.append(vgg.get_layer(layers).output)
  for layers in content_layers:
    content_outputs.append(vgg.get_layer(layers).output)

  model_output=style_outputs+content_outputs
  return models.Model(vgg.input,model_output)




In [ ]:
def content_loss(content,generated):
  return tf.reduce_mean(tf.square(content-generated))

In [ ]:
def gram_metrics(input):
  channels=input.shape[-1]
  input=tf.reshape(input,[-1,channels])
  n=tf.shape(input)[0]
  gram=tf.matmul(input,input,transpose_a=True)
  return gram/tf.cast(n,dtype=np.float32)

In [ ]:
p=gram_metrics(a)

In [ ]:
def style_loss(style,target_gram):
  style_gram=gram_metrics(style)
  return tf.reduce_mean(tf.square(style_gram-target_gram))

In [ ]:
style_loss(np.add(a,10),p)

In [ ]:
def get_features(model,content_path,style_path):
  content_image=load_and_process_image(content_path)
  style_image=load_and_process_image(style_path)
  style_outputs=model(style_image)
  content_outputs=model(content_image)
  style_features=[]
  content_features=[]
  for style_layers in style_outputs[:num_style_layers]:
    style_features.append(style_layers[0])
  for content_layers in content_outputs[num_style_layers:]:
    content_features.append(content_layers[0])

  return style_features,content_features

In [ ]:
def compute_cost(model,loss_weights,init_image,gram_style_features,content_features):
  style_weight,content_weight=loss_weights
  model_output=model(init_image)
  
  init_style_output_features=model_output[:num_style_layers]
  init_content_output_features=model_output[num_style_layers:]

  style_score=0
  content_score=0

  weight_per_style_layer=1.0/float(num_style_layers)

  for (output_per_layer,style_per_layer) in zip(init_style_output_features,gram_style_features):
    style_score+=weight_per_style_layer*style_loss(output_per_layer,style_per_layer)

  weights_per_content_layer=1.0/float(num_content_layers)

  for(output_per_layer,content_per_layer) in zip(init_content_output_features,content_features):
    content_score+=weights_per_content_layer*content_loss(output_per_layer,content_per_layer)

  style_score*=style_weight
  content_score*=content_weight

  loss=style_score+content_score

  return loss,style_score,content_score


In [ ]:
def compute_grads(cfg):
  with tf.GradientTape() as tape: 
    all_loss = compute_cost(**cfg)
  # Compute gradients wrt input image
  total_loss = all_loss[0]
  return tape.gradient(total_loss, cfg['init_image']), all_loss

In [ ]:
import IPython.display

def run_style_transfer(content_path, 
                       style_path,
                       num_iterations=1000,
                       content_weight=1e3, 
                       style_weight=1e-2): 
  # We don't need to (or want to) train any layers of our model, so we set their
  # trainable to false. 
  model = get_model() 
  for layer in model.layers:
    layer.trainable = False
  
  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_features(model, content_path, style_path)
  gram_style_features = [gram_metrics(style_feature) for style_feature in style_features]
  
  # Set initial image
  init_image = load_and_process_image(content_path)
  init_image = tf.Variable(init_image, dtype=tf.float64)
  # Create our optimizer
  #opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)
  opt=tf.keras.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

  # For displaying intermediate images 
  iter_count = 1
  
  # Store our best result
  best_loss, best_img = float('inf'), None
  
  # Create a nice config 
  loss_weights = (style_weight, content_weight)
  cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features
  }
    
  # For displaying
  num_rows = 2
  num_cols = 5
  display_interval = num_iterations/(num_rows*num_cols)
  start_time = time.time()
  global_start = time.time()
  
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   
  
  imgs = []
  for i in range(num_iterations):
    grads, all_loss = compute_grads(cfg)
    loss, style_score, content_score = all_loss
    opt.apply_gradients([(grads, init_image)])
    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time() 
    
    if loss < best_loss:
      # Update best loss and best image from total loss. 
      best_loss = loss
      best_img = deprocess_img(init_image.numpy())

    if i % display_interval== 0:
      start_time = time.time()
      
      # Use the .numpy() method to get the concrete numpy array
      plot_img = init_image.numpy()
      plot_img = deprocess_img(plot_img)
      imgs.append(plot_img)
      IPython.display.clear_output(wait=True)
      IPython.display.display_png(Image.fromarray(plot_img))
      print('Iteration: {}'.format(i))        
      print('Total loss: {:.4e}, ' 
            'style loss: {:.4e}, '
            'content loss: {:.4e}, '
            'time: {:.4f}s'.format(loss, style_score, content_score, time.time() - start_time))
  print('Total time: {:.4f}s'.format(time.time() - global_start))
  IPython.display.clear_output(wait=True)
  plt.figure(figsize=(14,4))
  for i,img in enumerate(imgs):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(img)
      plt.xticks([])
      plt.yticks([])
      
  return best_img, best_loss 

In [ ]:
best, best_loss = run_style_transfer(content_path, 
                                     style_path, num_iterations=1000)

In [ ]:
Image.fromarray(best)

In [ ]:
import matplotlib.image as mpimg
image=['/content/WIN_20200714_10_41_07_Pro.jpg','/content/style.jpg']
fig=plt.figure(figsize=(8, 8))
columns = 2
rows = 1
for i in range(1, columns*rows +1):
    img=mpimg.imread(image[i-1])
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
Image.fromarray(best)